# Proyecto Cambio de Divisas

In [26]:
!pip install sqlalchemy-redshift

In [27]:
#Cargando los paquetes
#Requests nos proporciona las capacidades para enviar una solicitud HTTP a un servidor.
import requests
import yaml
import json
import pytz
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text
import psycopg2

In [28]:
with open('apikey.yaml', 'r') as file:
    api_keys = yaml.safe_load(file)
api_key = api_keys['api_key']

### Obteniendo el cierre - código que se ejecutará diario

In [29]:
# Extraer la fecha actual para el horario de Mx
timezone = pytz.timezone('America/Mexico_City')
current_time = datetime.now(timezone)
formatted_date = current_time.strftime("%Y-%m-%d")

currencies_list = ['EUR', 'GBP', 'USD', 'PEN', 'BTC', 'KRW', 'INR', 'CNY', 'BRL', 'ARS', 'JPY']
currencies = ','.join(currencies_list)

base_url = f"https://api.apilayer.com/currency_data/timeframe?start_date={formatted_date}&end_date={formatted_date}&source=MXN&currencies={currencies}"

payload = {}
headers= {
  "apikey": api_key
}


response = requests.get(base_url, headers=headers, data=payload)
response.status_code

200

In [30]:
print(json.dumps(response.json(), indent=4))

{
    "success": true,
    "timeframe": true,
    "start_date": "2024-01-14",
    "end_date": "2024-01-14",
    "source": "MXN",
    "quotes": {
        "2024-01-14": {
            "MXNEUR": 0.054011,
            "MXNGBP": 0.046425,
            "MXNUSD": 0.059209,
            "MXNPEN": 0.218959,
            "MXNBTC": 1.374536e-06,
            "MXNKRW": 77.771942,
            "MXNINR": 4.90681,
            "MXNCNY": 0.421014,
            "MXNBRL": 0.287449,
            "MXNARS": 48.247876,
            "MXNJPY": 8.577352
        }
    }
}


In [31]:
datos = json.loads(response.text)

Este dataframe trae únicamente una fila, la cual es el valor actual del día en curso que se estará almacenando en la tabla

In [32]:
df = pd.DataFrame(datos)
df

,success,timeframe,start_date,end_date,source,quotes
2024-01-14,True,True,2024-01-14,2024-01-14,MXN,"{'MXNEUR': 0.054011, 'MXNGBP': 0.046425, 'MXNU..."


### Limpiando datos

In [33]:
df_quotes = pd.json_normalize(df['quotes'])


# Combinar las nuevas columnas con el DataFrame original
df = pd.concat([df, df_quotes], axis=1).drop('quotes', axis=1)

df

,success,timeframe,start_date,end_date,source,MXNEUR,MXNGBP,MXNUSD,MXNPEN,MXNBTC,MXNKRW,MXNINR,MXNCNY,MXNBRL,MXNARS,MXNJPY
2024-01-14,True,True,2024-01-14,2024-01-14,MXN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,0.054011,0.046425,0.059209,0.218959,0.000001,77.771942,4.90681,0.421014,0.287449,48.247876,8.577352


In [34]:
combined_row = df.iloc[0].combine_first(df.iloc[1])
# Convertir la serie combinada en un DataFrame de una sola fila
df_combined = pd.DataFrame([combined_row])
df_combined.rename(columns=lambda x: x.replace('MXN', '') if 'MXN' in x else x, inplace=True)
df_combined.rename(columns=lambda x: x.replace('start_date', 'date') if 'start_date' in x else x, inplace=True)
columns_to_drop = ['success', 'timeframe','end_date']
df_clean = df_combined.drop(columns=columns_to_drop).reset_index(drop=True)
df_clean

,date,source,EUR,GBP,USD,PEN,BTC,KRW,INR,CNY,BRL,ARS,JPY
0,2024-01-14,MXN,0.054011,0.046425,0.059209,0.218959,0.000001,77.771942,4.90681,0.421014,0.287449,48.247876,8.577352


In [35]:
# Conocer el tipo de dato en el df
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1 non-null      object 
 1   source  1 non-null      object 
 2   EUR     1 non-null      float64
 3   GBP     1 non-null      float64
 4   USD     1 non-null      float64
 5   PEN     1 non-null      float64
 6   BTC     1 non-null      float64
 7   KRW     1 non-null      float64
 8   INR     1 non-null      float64
 9   CNY     1 non-null      float64
 10  BRL     1 non-null      float64
 11  ARS     1 non-null      float64
 12  JPY     1 non-null      float64
dtypes: float64(11), object(2)
memory usage: 232.0+ bytes


### Cambiando tipo de datos

In [36]:
# Como se esta trabajando con fechas, es importante cambiar el tipo de dato de object a fecha
column = ['date']
df_clean[column] = df_clean[column].apply(pd.to_datetime, errors='coerce', axis=1)
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1 non-null      datetime64[ns]
 1   source  1 non-null      object        
 2   EUR     1 non-null      float64       
 3   GBP     1 non-null      float64       
 4   USD     1 non-null      float64       
 5   PEN     1 non-null      float64       
 6   BTC     1 non-null      float64       
 7   KRW     1 non-null      float64       
 8   INR     1 non-null      float64       
 9   CNY     1 non-null      float64       
 10  BRL     1 non-null      float64       
 11  ARS     1 non-null      float64       
 12  JPY     1 non-null      float64       
dtypes: datetime64[ns](1), float64(11), object(1)
memory usage: 232.0+ bytes


### Enviando datos a Redshift

In [37]:
# Cargar credenciales desde el archivo YAML
from sqlalchemy import inspect
with open('credentials.yaml', 'r') as file:
    credenciales = yaml.safe_load(file)['redshift']

# Utilizar credenciales ocultas
username = credenciales['username']
password = credenciales['password']
host = credenciales['host']
port = credenciales['port']
database = credenciales['database']

# Construir la cadena de conexión
cadena_conexion = f"postgresql://{username}:{password}@{host}:{port}/{database}"

# Conectar al motor de la base de datos
engine = create_engine(cadena_conexion)
df_clean.to_sql('cambios_divisa', engine, if_exists='append', index=False)


1

### Leer los datos de redshift que fueron cargador previamente (al día actual) + los que ya se habían cargado (histórico)

In [38]:
# Comprobar que los datos fueron cargados exitosamente en redshift al ser llamados
query = "SELECT * FROM cambios_divisa"
existing_data = pd.read_sql(query, engine)
existing_data

,date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy
0,2023-02-01,MXN,0.048826,0.043409,0.053770,0.206878,0.000002,65.721149,4.393538,0.362503,0.271805,10.068644,6.916104
1,2023-02-02,MXN,0.049154,0.043856,0.053601,0.204777,0.000002,65.674506,4.397749,0.360868,0.270722,10.054653,6.901198
2,2023-02-03,MXN,0.048691,0.043760,0.052721,0.201778,0.000002,65.788509,4.349079,0.357193,0.271648,9.881499,6.916796
3,2023-02-04,MXN,0.048691,0.043736,0.052721,0.203432,0.000002,65.788509,4.335471,0.357204,0.271611,9.881499,6.916023
4,2023-02-05,MXN,0.048787,0.043709,0.052644,0.203133,0.000002,65.691804,4.329098,0.356679,0.271211,9.902502,6.949845
...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2024-01-11,MXN,0.053871,0.046299,0.059157,0.218931,0.000001,77.715890,4.914539,0.420789,0.288165,48.239710,8.580532
345,2024-01-12,MXN,0.054079,0.046501,0.059284,0.219471,0.000001,77.869775,4.912981,0.421543,0.287775,48.308548,8.588138
346,2024-01-13,MXN,0.054079,0.046512,0.059284,0.219234,0.000001,77.869775,4.912981,0.421543,0.287811,48.308548,8.588138
347,2024-01-14,MXN,0.054079,0.046512,0.059284,0.219235,0.000001,77.870021,4.912998,0.421544,0.287811,48.308719,8.588166


### Puede haber duplicados en los datos previos, sin embargo, le pasamos un código para eliminarlos

In [39]:
# Concatenar los datos y eliminar duplicados
existing_data = existing_data.sort_values(by='date', ascending=False)
# Eliminar duplicados basados en la columna 'date' y conservar el primero (el más reciente)
df_combined = existing_data.groupby('date').first().reset_index()
df_combined

,date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy
0,2023-02-01,MXN,0.048826,0.043409,0.053770,0.206878,0.000002,65.721149,4.393538,0.362503,0.271805,10.068644,6.916104
1,2023-02-02,MXN,0.049154,0.043856,0.053601,0.204777,0.000002,65.674506,4.397749,0.360868,0.270722,10.054653,6.901198
2,2023-02-03,MXN,0.048691,0.043760,0.052721,0.201778,0.000002,65.788509,4.349079,0.357193,0.271648,9.881499,6.916796
3,2023-02-04,MXN,0.048691,0.043736,0.052721,0.203432,0.000002,65.788509,4.335471,0.357204,0.271611,9.881499,6.916023
4,2023-02-05,MXN,0.048787,0.043709,0.052644,0.203133,0.000002,65.691804,4.329098,0.356679,0.271211,9.902502,6.949845
...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,2024-01-10,MXN,0.053649,0.046199,0.058878,0.218290,0.000001,77.630311,4.888387,0.422237,0.288084,47.982977,8.580923
344,2024-01-11,MXN,0.053871,0.046299,0.059157,0.218931,0.000001,77.715890,4.914539,0.420789,0.288165,48.239710,8.580532
345,2024-01-12,MXN,0.054079,0.046501,0.059284,0.219471,0.000001,77.869775,4.912981,0.421543,0.287775,48.308548,8.588138
346,2024-01-13,MXN,0.054079,0.046512,0.059284,0.219234,0.000001,77.869775,4.912981,0.421543,0.287811,48.308548,8.588138


In [40]:
# Cambiar tipo de datos
columnas = ['date']
df_combined[columnas] = df_combined[columnas].apply(pd.to_datetime, errors='coerce', axis=1)
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    348 non-null    datetime64[ns]
 1   source  348 non-null    object        
 2   eur     348 non-null    float64       
 3   gbp     348 non-null    float64       
 4   usd     348 non-null    float64       
 5   pen     348 non-null    float64       
 6   btc     348 non-null    float64       
 7   krw     348 non-null    float64       
 8   inr     348 non-null    float64       
 9   cny     348 non-null    float64       
 10  brl     348 non-null    float64       
 11  ars     348 non-null    float64       
 12  jpy     348 non-null    float64       
dtypes: datetime64[ns](1), float64(11), object(1)
memory usage: 35.5+ KB


### Volver a cargar datos pero ahora evitando duplicados

In [41]:
# Escribir los datos combinados y sin duplicados de nuevo en la base de datos
df_combined.to_sql('cambios_divisa', engine, if_exists='replace', index=False)
df_combined

,date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy
0,2023-02-01,MXN,0.048826,0.043409,0.053770,0.206878,0.000002,65.721149,4.393538,0.362503,0.271805,10.068644,6.916104
1,2023-02-02,MXN,0.049154,0.043856,0.053601,0.204777,0.000002,65.674506,4.397749,0.360868,0.270722,10.054653,6.901198
2,2023-02-03,MXN,0.048691,0.043760,0.052721,0.201778,0.000002,65.788509,4.349079,0.357193,0.271648,9.881499,6.916796
3,2023-02-04,MXN,0.048691,0.043736,0.052721,0.203432,0.000002,65.788509,4.335471,0.357204,0.271611,9.881499,6.916023
4,2023-02-05,MXN,0.048787,0.043709,0.052644,0.203133,0.000002,65.691804,4.329098,0.356679,0.271211,9.902502,6.949845
...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,2024-01-10,MXN,0.053649,0.046199,0.058878,0.218290,0.000001,77.630311,4.888387,0.422237,0.288084,47.982977,8.580923
344,2024-01-11,MXN,0.053871,0.046299,0.059157,0.218931,0.000001,77.715890,4.914539,0.420789,0.288165,48.239710,8.580532
345,2024-01-12,MXN,0.054079,0.046501,0.059284,0.219471,0.000001,77.869775,4.912981,0.421543,0.287775,48.308548,8.588138
346,2024-01-13,MXN,0.054079,0.046512,0.059284,0.219234,0.000001,77.869775,4.912981,0.421543,0.287811,48.308548,8.588138
